# Analyze types of MPA

In [7]:
import geojson
import pandas as pd
import numpy as np

In [8]:
# Load geojson file
path = '../assets/California_Marine_Protected_Areas_[ds582].geojson'
with open(path) as f:
    gj = geojson.load(f)

# Extract Type, FULLNAME, OBJECTID
dict_MPA = {'OBJECTID':[], 'Type':[], 'FULLNAME':[]}
for i in range(len(gj['features'])):
    for key in dict_MPA.keys():
        dict_MPA[key].append(gj['features'][i]['properties'][key])
        
# Create dataframe from dict_MPA
df_MPA = pd.DataFrame.from_dict(dict_MPA)

In [25]:
pd.unique(df_MPA['Type'])

array(['SMCA', 'Special Closure', 'SMR', 'SMRMA', 'SMP', 'SMCA (No-Take)',
       'FMR', 'FMCA'], dtype=object)

In [26]:
df_MPA['Type'].value_counts()

SMCA               61
SMR                49
Special Closure    14
SMCA (No-Take)     10
FMR                 8
SMP                 7
SMRMA               5
FMCA                1
Name: Type, dtype: int64

# Extract MPA regulations from txt

In [57]:
import json

mpa_regulations = {}
name = ""
regulations = []
item = ""

with open('data.txt', 'r') as f:
    for i, line in enumerate(f):
        
        if line[0] == '(':
            
            if item:
                mpa_regulations[name].append(item)
                item = ''
            
            if line[1].isdigit():
                name = line.split(' ', 1)[1].rstrip()    # Remove newline
                key_sentence = 'Special restrictions'
                index = line.find(key_sentence)
                if index != -1:
                    name = name[:index-2]    # Remove Special restrictions...
                else:
                    name = name[:-1]    # Remove dot
                mpa_regulations[name] = []

            elif line[1] == 'B':
                key_sentence = 'with the following specified exceptions:'
                index = line.find(key_sentence)
                # Thers is sentence after 'with the following specified exceptions:'
                if index != -1 and index + len(key_sentence) + 1 < len(line):
                    mpa_regulations[name].append(line[index + len(key_sentence) + 1:].rstrip())
                elif index == -1:
                    mpa_regulations[name].append(line[4:].rstrip())
            elif line[1] != 'A':
                mpa_regulations[name].append(line.split(' ', 1)[1].rstrip())

        elif (line[0].isdigit() and line[1]=='.'):
            if item:
                mpa_regulations[name].append(item)
                
            item = line.split(' ', 1)[1].rstrip()
        
        elif item:
            item += ' ' + line.rstrip()


In [59]:
json_path = 'mpa_regulations_new.json'
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(mpa_regulations, f, ensure_ascii=False, indent=4)
